# A NOTEBOOK TO REPLICATE APPROACH OF HANGGI ET AL. 2011
## Cortical thickness
Thresholded correlation matrices generated in R.

In [1]:
# Import packages and scripts
import itertools
import sys
from pathlib import Path
import pandas as pd
import scipy
from graph_tool.all import *
import matplotlib.pyplot as plt
import matplotlib.cm
import numpy as np
from os import listdir
import re
import scipy.stats as stats
from IPython.display import display
from PIL import Image

common_path = Path("/workspace/scripts/common")
sys.path.append(str(common_path))
import networkfuncs # import networkfuncs script

In [69]:
# Import matrices
datapath = '/workspace/shared/synesthesia_100brains/thickness/hanggi_replication/' # set path to data
files = listdir(datapath) # get files
mats = [str(file) for file in files if re.search('adj_mat_0', file)] # get control subsamples
graphs = [] 
for i in range(len(mats)):
    data = pd.read_csv(datapath+mats[i], index_col=0)
    graphs.append(Graph(scipy.sparse.lil_matrix(data), directed = False))
    remove_self_loops(graphs[i])

In [71]:
graphs

[<Graph object, undirected, with 360 vertices and 50280 edges, 1 internal edge property, at 0x7f93160bec30>,
 <Graph object, undirected, with 360 vertices and 55075 edges, 1 internal edge property, at 0x7f9314802030>,
 <Graph object, undirected, with 360 vertices and 46473 edges, 1 internal edge property, at 0x7f931cd42ed0>,
 <Graph object, undirected, with 360 vertices and 52237 edges, 1 internal edge property, at 0x7f931699b3e0>,
 <Graph object, undirected, with 360 vertices and 37416 edges, 1 internal edge property, at 0x7f93159a55e0>,
 <Graph object, undirected, with 360 vertices and 45184 edges, 1 internal edge property, at 0x7f931598c230>,
 <Graph object, undirected, with 360 vertices and 32799 edges, 1 internal edge property, at 0x7f93148038c0>,
 <Graph object, undirected, with 360 vertices and 41156 edges, 1 internal edge property, at 0x7f931cd420c0>,
 <Graph object, undirected, with 360 vertices and 28143 edges, 1 internal edge property, at 0x7f931d019250>,
 <Graph object, und

In [72]:
# Get network metrics
net_metrics = [] # initialise metric list
for graph, group_name in zip(graphs, ['Control', 'Syn'] * 16): # for each graph and group
    net_metric = networkfuncs.measure_net(graph) # get the network metrics
    df = net_metric[0] # get metrics
    df['Group'] = group_name # get the group name
    net_metrics.append(df) # add net metrics to list
    
net_metrics = pd.concat(net_metrics, ignore_index=True) # turn network metrics into one df
thresholds = [re.search(r'\d\.\d+', mat).group() for mat in mats]
net_metrics['threshold'] = thresholds

print(net_metrics) # print results
    

    clustering  efficiency     L_obs   mean_eb   mean_vb    Group threshold
0     0.300809    0.289474  1.917499  0.000029  0.001230  Control      0.15
1     0.346421    0.324829  1.712265  0.000024  0.000947      Syn      0.15
2     0.299829    0.289462  1.917750  0.000031  0.001233  Control     0.175
3     0.346454    0.324827  1.712315  0.000026  0.000949      Syn     0.175
4     0.296542    0.289044  1.923871  0.000039  0.001314  Control     0.225
5     0.345270    0.324636  1.714898  0.000030  0.000987      Syn     0.225
6     0.295020    0.287853  1.942611  0.000046  0.001459  Control      0.25
7     0.343912    0.324103  1.720864  0.000034  0.001069      Syn      0.25
8     0.293446    0.284518       inf  0.000055  0.001591  Control     0.275
9     0.342354    0.322761  1.732657  0.000039  0.001219      Syn     0.275
10    0.298176    0.289386  1.919149  0.000034  0.001251  Control       0.2
11    0.346154    0.324793  1.712860  0.000027  0.000956      Syn       0.2
12    0.2915

In [45]:
# Get randomised graph metrics
def process_graph(graph, num_rewires=100):
    net_metrics_list = [] # initialise metrics for each random graph
    for _ in range(num_rewires): # for each rewire
        graph_copy = graph.copy() # copy graph
        random_rewire(graph_copy, model="configuration") # randomise graph but retaining degree sequence 
        net_metrics_list.append(networkfuncs.measure_net(graph_copy)[0])  # get metrics of random graph
        
    net_metrics_mean = pd.concat(net_metrics_list).mean().to_frame().T  # get mean over all rewires
    return net_metrics_mean # return

net_metrics_rand = pd.concat([process_graph(graph) for graph in graphs], ignore_index=True)

In [50]:
# Add gamma, lambda and sigma to results
net_metrics['gamma'] = net_metrics['clustering'] / net_metrics_rand['clustering']
net_metrics['lambda'] = net_metrics['L_obs'] / net_metrics_rand['L_obs']
net_metrics['sigma'] = net_metrics['gamma'] / net_metrics['lambda']

print(net_metrics)

    clustering  efficiency     L_obs   mean_eb   mean_vb    Group threshold  \
0     0.282975    0.306802  1.740723  0.000030  0.001415  Control      0.15   
1     0.337185    0.335542  1.608988  0.000025  0.001063      Syn      0.15   
2     0.276209    0.309308  1.720534  0.000033  0.001472  Control     0.175   
3     0.333971    0.337098  1.596114  0.000027  0.001094      Syn     0.175   
4     0.258915    0.312436  1.697568  0.000042  0.001562  Control     0.225   
5     0.324312    0.340068  1.574430  0.000032  0.001191      Syn     0.225   
6     0.251266    0.312228  1.700430  0.000048  0.001604  Control      0.25   
7     0.317842    0.341087  1.567961  0.000035  0.001244      Syn      0.25   
8     0.243577    0.310648  1.715107  0.000057  0.001691  Control     0.275   
9     0.310955    0.341805  1.563027  0.000040  0.001315      Syn     0.275   
10    0.267713    0.310889  1.710615  0.000037  0.001523  Control       0.2   
11    0.329703    0.339051  1.581668  0.000029  0.00

In [51]:
net_metrics.to_csv(datapath+'hanggi_original_thresholds_net_metrics.csv', index=False)

## Repeat for different thresholds
Since using the original thresholds left some nodes disconnected, let's now repeat the process but using some other thresholds. Below, we have shifted the thresholds to now go up to the threshold that caused node disconnection previously; however, we have retained the same intervals as before.

In [73]:
# Import matrices
datapath = '/workspace/shared/synesthesia_100brains/thickness/hanggi_replication/' # set path to data
files = listdir(datapath) # get files
mats = [str(file) for file in files if re.search('adj_mat_shift', file)] # get control subsamples
graphs = [] 
for i in range(len(mats)):
    data = pd.read_csv(datapath+mats[i], index_col=0)
    graphs.append(Graph(scipy.sparse.lil_matrix(data), directed = False))
    remove_self_loops(graphs[i])

In [75]:
# Get network metrics
net_metrics = [] # initialise metric list
for graph, group_name in zip(graphs, ['Control', 'Syn'] * 10): # for each graph and group
    net_metric = networkfuncs.measure_net(graph) # get the network metrics
    df = net_metric[0] # get metrics
    df['Group'] = group_name # get the group name
    net_metrics.append(df) # add net metrics to list
    
net_metrics = pd.concat(net_metrics, ignore_index=True) # turn network metrics into one df
thresholds = [re.search(r'\d\.\d+', mat).group() for mat in mats]
net_metrics['threshold'] = thresholds

print(net_metrics) # print results
    

    clustering  efficiency     L_obs   mean_eb   mean_vb    Group threshold
0     0.301397    0.289477  1.917412  0.000023  0.001229  Control     0.025
1     0.345402    0.324830  1.712256  0.000021  0.000947      Syn     0.025
2     0.301513    0.289477  1.917412  0.000024  0.001229  Control      0.05
3     0.345515    0.324830  1.712256  0.000022  0.000947      Syn      0.05
4     0.301630    0.289477  1.917412  0.000025  0.001229  Control     0.075
5     0.345711    0.324830  1.712256  0.000022  0.000947      Syn     0.075
6     0.301438    0.289477  1.917412  0.000027  0.001229  Control     0.125
7     0.346264    0.324830  1.712256  0.000023  0.000947      Syn     0.125
8     0.300809    0.289474  1.917499  0.000029  0.001230  Control      0.15
9     0.346421    0.324829  1.712265  0.000024  0.000947      Syn      0.15
10    0.299829    0.289462  1.917750  0.000031  0.001233  Control     0.175
11    0.346454    0.324827  1.712315  0.000026  0.000949      Syn     0.175
12    0.3016

In [76]:
# Get randomised graph metrics
def process_graph(graph, num_rewires=100):
    net_metrics_list = [] # initialise metrics for each random graph
    for _ in range(num_rewires): # for each rewire
        graph_copy = graph.copy() # copy graph
        random_rewire(graph_copy, model="configuration") # randomise graph but retaining degree sequence 
        net_metrics_list.append(networkfuncs.measure_net(graph_copy)[0])  # get metrics of random graph
        
    net_metrics_mean = pd.concat(net_metrics_list).mean().to_frame().T  # get mean over all rewires
    return net_metrics_mean # return

net_metrics_rand = pd.concat([process_graph(graph) for graph in graphs], ignore_index=True)

In [77]:
# Add gamma, lambda and sigma to results
net_metrics['gamma'] = net_metrics['clustering'] / net_metrics_rand['clustering']
net_metrics['lambda'] = net_metrics['L_obs'] / net_metrics_rand['L_obs']
net_metrics['sigma'] = net_metrics['gamma'] / net_metrics['lambda']

print(net_metrics)

    clustering  efficiency     L_obs   mean_eb   mean_vb    Group threshold  \
0     0.301397    0.289477  1.917412  0.000023  0.001229  Control     0.025   
1     0.345402    0.324830  1.712256  0.000021  0.000947      Syn     0.025   
2     0.301513    0.289477  1.917412  0.000024  0.001229  Control      0.05   
3     0.345515    0.324830  1.712256  0.000022  0.000947      Syn      0.05   
4     0.301630    0.289477  1.917412  0.000025  0.001229  Control     0.075   
5     0.345711    0.324830  1.712256  0.000022  0.000947      Syn     0.075   
6     0.301438    0.289477  1.917412  0.000027  0.001229  Control     0.125   
7     0.346264    0.324830  1.712256  0.000023  0.000947      Syn     0.125   
8     0.300809    0.289474  1.917499  0.000029  0.001230  Control      0.15   
9     0.346421    0.324829  1.712265  0.000024  0.000947      Syn      0.15   
10    0.299829    0.289462  1.917750  0.000031  0.001233  Control     0.175   
11    0.346454    0.324827  1.712315  0.000026  0.00

In [78]:
net_metrics.to_csv(datapath+'hanggi_shifted_thresholds_net_metrics.csv', index=False)

## Adjusted thresholds
The shifted thresholds don't have much change for the first half of threshold values. Here we have adjusted the thresholds to cover a smaller range, taking smaller steps

In [4]:
# Import matrices
datapath = '/workspace/shared/synesthesia_100brains/thickness/hanggi_replication/'  # set path to data
files = listdir(datapath)  # get files
mats = [str(file) for file in files if re.search('adj_mat_adjust', file)]  # get control subsamples
graphs = []
for i in range(len(mats)):
    data = pd.read_csv(datapath + mats[i], index_col=0)
    graphs.append(Graph(scipy.sparse.lil_matrix(data), directed=False))
    remove_self_loops(graphs[i])

In [5]:
# Get network metrics
net_metrics = []  # initialise metric list
for graph, group_name in zip(graphs, ['Control', 'Syn'] * 11):  # for each graph and group
    net_metric = networkfuncs.measure_net(graph)  # get the network metrics
    df = net_metric[0]  # get metrics
    df['Group'] = group_name  # get the group name
    net_metrics.append(df)  # add net metrics to list

net_metrics = pd.concat(net_metrics, ignore_index=True)  # turn network metrics into one df
thresholds = [re.search(r'\d\.\d+', mat).group() for mat in mats]
net_metrics['threshold'] = thresholds

print(net_metrics)  # print results

    clustering  efficiency     L_obs   mean_eb   mean_vb    Group threshold
0     0.301438    0.289477  1.917412  0.000027  0.001229  Control     0.125
1     0.346264    0.324830  1.712256  0.000023  0.000947      Syn     0.125
2     0.301216    0.289476  1.917444  0.000028  0.001229  Control    0.1375
3     0.346377    0.324829  1.712257  0.000024  0.000947      Syn    0.1375
4     0.300809    0.289474  1.917499  0.000029  0.001230  Control      0.15
5     0.346421    0.324829  1.712265  0.000024  0.000947      Syn      0.15
6     0.300336    0.289470  1.917585  0.000030  0.001231  Control    0.1625
7     0.346408    0.324828  1.712281  0.000025  0.000948      Syn    0.1625
8     0.299829    0.289462  1.917750  0.000031  0.001233  Control     0.175
9     0.346454    0.324827  1.712315  0.000026  0.000949      Syn     0.175
10    0.299073    0.289432  1.918415  0.000033  0.001240  Control    0.1875
11    0.346409    0.324817  1.712484  0.000026  0.000951      Syn    0.1875
12    0.2974

In [6]:
# Get randomised graph metrics
def process_graph(graph, num_rewires=100):
    net_metrics_list = [] # initialise metrics for each random graph
    for _ in range(num_rewires): # for each rewire
        graph_copy = graph.copy() # copy graph
        random_rewire(graph_copy, model="configuration") # randomise graph but retaining degree sequence 
        net_metrics_list.append(networkfuncs.measure_net(graph_copy)[0])  # get metrics of random graph
        
    net_metrics_mean = pd.concat(net_metrics_list).mean().to_frame().T  # get mean over all rewires
    return net_metrics_mean # return

net_metrics_rand = pd.concat([process_graph(graph) for graph in graphs], ignore_index=True)

In [7]:
# Add gamma, lambda and sigma to results
net_metrics['gamma'] = net_metrics['clustering'] / net_metrics_rand['clustering']
net_metrics['lambda'] = net_metrics['L_obs'] / net_metrics_rand['L_obs']
net_metrics['sigma'] = net_metrics['gamma'] / net_metrics['lambda']

print(net_metrics)

    clustering  efficiency     L_obs   mean_eb   mean_vb    Group threshold  \
0     0.301438    0.289477  1.917412  0.000027  0.001229  Control     0.125   
1     0.346264    0.324830  1.712256  0.000023  0.000947      Syn     0.125   
2     0.301216    0.289476  1.917444  0.000028  0.001229  Control    0.1375   
3     0.346377    0.324829  1.712257  0.000024  0.000947      Syn    0.1375   
4     0.300809    0.289474  1.917499  0.000029  0.001230  Control      0.15   
5     0.346421    0.324829  1.712265  0.000024  0.000947      Syn      0.15   
6     0.300336    0.289470  1.917585  0.000030  0.001231  Control    0.1625   
7     0.346408    0.324828  1.712281  0.000025  0.000948      Syn    0.1625   
8     0.299829    0.289462  1.917750  0.000031  0.001233  Control     0.175   
9     0.346454    0.324827  1.712315  0.000026  0.000949      Syn     0.175   
10    0.299073    0.289432  1.918415  0.000033  0.001240  Control    0.1875   
11    0.346409    0.324817  1.712484  0.000026  0.00

In [8]:
net_metrics.to_csv(datapath+'hanggi_adjusted_thresholds_net_metrics.csv', index=False)